In [89]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

### WebScraping 

In [ ]:
scraper = cloudscraper.create_scraper()
headers = {

'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

}
dados_rj = ['https://www.zapimoveis.com.br/aluguel/apartamentos/rj+rio-de-janeiro/?onde=%2CRio+de+Janeiro%2CRio+de+Janeiro%2C%2C%2C%2C%2Ccity%2CBR>Rio+de+Janeiro>NULL>Rio+de+Janeiro%2C-22.906847%2C-43.172897%2C&tipos=apartamento_residencial&pagina=1&transacao=aluguel'] + \
             ['https://www.zapimoveis.com.br/aluguel/apartamentos/rj+rio-de-janeiro/?onde=%2CRio+de+Janeiro%2CRio+de+Janeiro%2C%2C%2C%2C%2Ccity%2CBR>Rio+de+Janeiro>NULL>Rio+de+Janeiro%2C-22.906847%2C-43.172897%2C&tipos=apartamento_residencial&pagina={}&transacao=aluguel'.format(i) for i in range(2, 323)]



In [ ]:
data = []
for dados in dados_rj:
    response = scraper.get(dados, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    pagina_atual = soup.find_all('li', {'data-cy' : 'rp-property-cd'})
    
    for item in pagina_atual:
        local_element = item.find('h2', {'data-cy': 'rp-cardProperty-location-txt'})
        local = local_element.text.strip() if local_element else 'NA'

        area_element = item.find('li', {'data-cy': 'rp-cardProperty-propertyArea-txt'})
        area = area_element.text.strip() if area_element else 'NA'
        
        quartos_element = item.find('li', {'data-cy': 'rp-cardProperty-bedroomQuantity-txt'})
        quartos = quartos_element.text.strip() if quartos_element else 'NA'
        
        banheiros_element = item.find('li', {'data-cy': 'rp-cardProperty-bathroomQuantity-txt'})
        banheiros = banheiros_element.text.strip() if banheiros_element else 'NA'
        
        vagas_element = item.find('li', {'data-cy': 'rp-cardProperty-parkingSpacesQuantity-txt'})
        vagas = vagas_element.text.strip() if vagas_element else 'NA' 

        preco_element = item.find('div', {'data-cy' : 'rp-cardProperty-price-txt'})
        preco = preco_element.text.strip() if preco_element else 'NA'
        
        link_element = item.find('a', href=True)
        link = None
        if link_element and link_element['href'].startswith('https://www.zapimoveis.com.br/imovel/'):
            link = link_element['href']

        data.append({
            'Local': local, 
            'Preço': preco,
            'Área': area,
            'Quartos': quartos,
            'Banheiros': banheiros,
            'Vagas': vagas,
            'Link': link
        })
    
    time.sleep(1) 

In [ ]:

database = pd.DataFrame(data)
database.to_csv('imoveis_rio.csv', index = False)

### Ajustando a base de dados

In [121]:
db = pd.read_csv('imoveis_rio.csv', sep=',')
db['Bairro'] = 0
db['Aluguel(R$)'] = 0
db['Condomínio(R$)'] = 0
db['IPTU(R$)'] = 0
db.head()

,Local,Preço,Área,Quartos,Banheiros,Vagas,Link,Bairro,Aluguel(R$),Condomínio(R$),IPTU(R$)
0,"Apartamento para alugar em Barra da Tijuca, Ri...",R$ 9.650/mêsCond. R$ 7.322 • IPTU R$ 18.972,Tamanho do imóvel 99 m²,Quantidade de quartos 2,Quantidade de banheiros 3,Quantidade de vagas de garagem 1,https://www.zapimoveis.com.br/imovel/venda-apa...,0,0,0,0
1,"Apartamento para alugar em Leblon, Rio de Janeiro",R$ 9.000/mêsCond. R$ 2.250 • IPTU R$ 815,Tamanho do imóvel 94 m²,Quantidade de quartos 2,Quantidade de banheiros 3,Quantidade de vagas de garagem 1,https://www.zapimoveis.com.br/imovel/aluguel-a...,0,0,0,0
2,"Apartamento para alugar em Ipanema, Rio de Jan...",R$ 40.000/mêsCond. R$ 4.438 • IPTU R$ 3.711,Tamanho do imóvel 280 m²,Quantidade de quartos 4,Quantidade de banheiros 1,Quantidade de vagas de garagem 3,https://www.zapimoveis.com.br/imovel/aluguel-a...,0,0,0,0
3,"Apartamento para alugar em Barra da Tijuca, Ri...",R$ 4.500/mêsCond. R$ 1.478 • IPTU R$ 299,Tamanho do imóvel 61 m²,Quantidade de quartos 2,Quantidade de banheiros 2,Quantidade de vagas de garagem 1,https://www.zapimoveis.com.br/imovel/aluguel-a...,0,0,0,0
4,"Apartamento para alugar em Copacabana, Rio de ...",R$ 14.000/mêsCond. R$ 3.200 • IPTU R$ 1.400,Tamanho do imóvel 298-300 m²,Quantidade de quartos 4,Quantidade de banheiros 2-4,Quantidade de vagas de garagem 1-4,NaN,0,0,0,0


In [123]:
db['Bairro'] = db['Local'].str.extract(r'em\s+([^,]+),')
db['Aluguel(R$)'] = db['Preço'].str.extract(r'R\$\s*([\d\.]+(?:\/(?:mês|dia))?)')
db['IPTU(R$)'] = db['Preço'].str.extract(r'IPTU\s*R\$\s*([\d\.]+)')
db['Condomínio(R$)'] = db['Preço'].str.extract(r'Cond\.\s*R\$\s*([\d\.]+)')
db['Área'] = db['Área'].str.split()
db['Área'] = db['Área'].str[-2]
db['Área'].rename('Área(m²)')
db['Quartos'] = db['Quartos'].str.split()
db['Quartos'] = db['Quartos'].str[-1]
db['Banheiros'] = db['Banheiros'].str.split()
db['Banheiros'] = db['Banheiros'].str[-1]
db['Vagas'] = db['Vagas'].str.split()
db['Vagas'] = db['Vagas'].str[-1]

db.head()

,Local,Preço,Área,Quartos,Banheiros,Vagas,Link,Bairro,Aluguel(R$),Condomínio(R$),IPTU(R$)
0,"Apartamento para alugar em Barra da Tijuca, Ri...",R$ 9.650/mêsCond. R$ 7.322 • IPTU R$ 18.972,NaN,2,3,1,https://www.zapimoveis.com.br/imovel/venda-apa...,Barra da Tijuca,9.650/mês,7.322,18.972
1,"Apartamento para alugar em Leblon, Rio de Janeiro",R$ 9.000/mêsCond. R$ 2.250 • IPTU R$ 815,NaN,2,3,1,https://www.zapimoveis.com.br/imovel/aluguel-a...,Leblon,9.000/mês,2.250,815
2,"Apartamento para alugar em Ipanema, Rio de Jan...",R$ 40.000/mêsCond. R$ 4.438 • IPTU R$ 3.711,NaN,4,1,3,https://www.zapimoveis.com.br/imovel/aluguel-a...,Ipanema,40.000/mês,4.438,3.711
3,"Apartamento para alugar em Barra da Tijuca, Ri...",R$ 4.500/mêsCond. R$ 1.478 • IPTU R$ 299,NaN,2,2,1,https://www.zapimoveis.com.br/imovel/aluguel-a...,Barra da Tijuca,4.500/mês,1.478,299
4,"Apartamento para alugar em Copacabana, Rio de ...",R$ 14.000/mêsCond. R$ 3.200 • IPTU R$ 1.400,NaN,4,2-4,1-4,NaN,Copacabana,14.000/mês,3.200,1.400
